## Project 

In this project we aim to develop an **estimate of velocity and position** from **GPS** data which is normally **noisy**.

Kalman filter offers a solution to the problem of processing the gps data and estimation of the state 

state of the vehicle can be defined as a vector $v_{x},v_{y},x,y$

## What do we need ? 

To implement kalman filter, we need two equations 

- Equation to describe the state transition behaviour, how is my state transitioning over time ?
- Model for the measurement process, how the measured data can be mapped to a state space !

state = $[x,y,v_{x},v_{y}]$

### State Transition 



 $$ state(t) = fn(state(t-1),control,noise) $$

 suppose that we replace state by x 

 $$ x(t) = fn(x(t-1),\epsilon) $$

to figure out the exact equation, let's try to resolve each term transion at it's own

$$v_{x}(t) = v_{x}(t-1)  + a_{x} \Delta t$$
$$v_{y}(t) = v_{y}(t-1)  + a_{y} \Delta t$$

$$ x(t) = x(t-1) + tv_{x}+a_{x}t^{2}$$

$$ y(t) = y(t-1) + tv_{y}+a_{y}t^{2}$$


### State Transition in Vector Form



# Project Implementation
## step 1 - Initializting the prediction 

In this project, we start first by state initialization using the stream operator `<<`, since we already know that the robot starts moving with 5 m/s at an angle of 45 degrees:

$$
\mathbf{state}_{t_0} =
\begin{bmatrix}
x \\
y \\
v_x \\
v_y
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0 \\
5 \cos(45^\circ) \\
5 \sin(45^\circ)
\end{bmatrix}
$$

## Implementation in C++ (Eigen)

We use the `<<` stream operator to fill the vector in a quicker way:

```cpp
#include <Eigen/Dense>
#include <cmath>
using namespace Eigen;

void KalmanFilter::predictionStep(double dt){
    if (!isInitialised() && INIT_ON_FIRST_PREDICTION)
    {
        Vector4d state = Vector4d::Zero();
        state << 0, 0, 5.0*cos(M_PI/4), 5.0*sin(M_PI/4);

        setState(state);

    }

}

```

### what is Eigen ? 
Eigen is a C++ template library for linear algebra, widely used for handling vectors, matrices, and related operations efficiently. It’s especially popular in robotics, physics simulations, computer graphics, and machine learning.

## Result  ? 

Just basic state initialization, if we build the simualtion without any modfication for this state over time, we will end up with growing estimation error which can be seen from the animation below.
<div style="text-align: center;">
    <img src="Figures/1.gif" width="400"/>
</div>


In summary we have a good initialization, but yet we don't update the predicted state properly as expected over time steps, an update step have to accomedate for the system and update correctly over time.


## Step 2 - Using a mathmatical model to predict motion over time.

lets assume that the mean of the state will progress over time throught the state transition matrix (stm)

in that cae : 
$$ state = state(t-1) * stm $$
$$
\mathbf{stm} =
\begin{bmatrix}
\overset{\raisebox{1em}{$x$}}{1} & 
\overset{\raisebox{1em}{$y$}}{0} & 
\overset{\raisebox{1em}{$v_x$}}{dt} & 
\overset{\raisebox{1em}{$v_y$}}{0} \\
0 & 1 & 0 & dt \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$


## Step 2 - Implementation in C++ (Eigen)

We use the `<<` stream operator to fill the vector in a quicker way:

```cpp
#include <Eigen/Dense>
#include <cmath>
using namespace Eigen;

void KalmanFilter::predictionStep(double dt){
    if (isInitialised())
    {
        MatrixXd stm = Matrix4d();
        stm << 
        1,0,dt,0,
        0,1,0,dt,
        0,0,1,0,
        0,0,0,1;


        state = stm * state;
        setState(state);

    }

}

```

## Result  Step  -  2 

The vehicle have a more porper estimate of it's location over time, by using the mathmatical model to update each time step, resulting an overall zero error ! 



<div style="text-align: center;">
    <img src="Figures/2.gif" width="400"/>
</div>



## Step 3 - Acommodate for uncertanity in mathamtical model 

Right now our framwork doesn't allow for any error  estimations, errors doesn't grow or die over time, we don't need any measurements, and all is good 


however this doesn't capture the reality, our initial state might not be that accurate, the mathmatical model that govern state transition might not be that good, anything can go wrong.


To allow for errors, we need to start by setting an unceratnity value of our progress overtime (acceleration), we still could have added the error any where !.


we start by initializing a covariance matrix to represnet uncertanity, since we are tracking state composed of four different variables, uncertatnity will invlovle all of them.


You will need to make 2 different modification, first one is the initializaiton to make sure we have an initalized covariance matrix. 


second step, we introduce a way to move the covariance over time, by using the same transition matrix.

### why do we need to update the covariance matrix ? 

- covariance matrix is time agnostic, it is not aware of the current time.

- let's assume that we won't progress it over time and we live in 1 dimenstional world, we start from a point x = 1 and guassian distruibtion around it with certain limits for example 1 have undertanity of 0.1 meaning it's either we stand at 1.1, 0.9 or something in between 

- in the second time step we stand at 2 but definitly even if we didn't add any noise we can't say that we are only in between 1.9 and 2.1 why ? basically this ignores the previous uncertanity that we had in the previous time step in the veloicty, starting from 0,9 means that we might end up in earlier point eg: 1.8. especially if we move slowly or faster that estimated

- this means that we must update the covariance matrix, in other words, if we move infinite amount of steps our uncertanity should end up covering from negative to positive infinity not around the infinity point with positive and negative 0.1 ! 

```cpp
#include <Eigen/Dense>
#include <cmath>
using namespace Eigen;

void KalmanFilter::predictionStep(double dt){
    if (isInitialised())
    {
        MatrixXd cov = Matrix4d::Zero();
        cov(0,0) = 0.01;
        cov(1,1) = 0.01;
        cov(2,2) = 0.01;
        cov(3,3) = 0.01;

        cov = stm*cov*stm.transpose();




        MatrixXd stm = Matrix4d();
        stm << 
        1,0,dt,0,
        0,1,0,dt,
        0,0,1,0,
        0,0,0,1;





        state = stm * state;
        setState(state);
        setCovariance(cov);

    }



}

```

## Result  Step  -  3 

Results below shows how uncertanity update is essential to accomedate for uncertnatiy in time step 2 with time step 1, making the undertanity circle grows indifiently.


<div style="display: flex; justify-content: space-between;">
    <img src="figures/3.gif" width="400"/>
    <img src="figures/4.gif" width="400"/>
</div>



## Step 4 - Constnat Velocity  Vs the tricky wind 

up till now we have reached a good point in estimating the velocity and the location given that we are somehow certain that object move with zero acceleration.

Let's say we live an a guassian planet, where wind hits in a guassian distribution, to accomodate for this in our model we must modify the state vecotr to have part for the acceleration, but the wind must not only be guassian but also with zero distribution.

In this way we can introduce it as noise not affecting the state but only resulting shift in the position that doesn't need to be carried over throught nonlinearly but carried over throught velocity only!.

Another way to think of this as a way to give acceleration to the boundary of the red estimation growth instead of growing normally it now have accelartion of growth too.


```cpp
#include <Eigen/Dense>
#include <cmath>
using namespace Eigen;

void KalmanFilter::predictionStep(double dt){
    if (isInitialised())
    {
        MatrixXd Q = Matrix2d::Zero(); 
        Q(0,0) = 1;
        Q(1,1) = 1;




        MatrixXd cov = Matrix4d::Zero();
        cov(0,0) = 0.01;
        cov(1,1) = 0.01;
        cov(2,2) = 0.01;
        cov(3,3) = 0.01;

        cov = stm*cov*stm.transpose();




        MatrixXd stm = Matrix4d();
        stm << 
        1,0,dt,0,
        0,1,0,dt,
        0,0,1,0,
        0,0,0,1;




        cov =  stm * cov * stm.transpose()+ L*Q*L.transpose();
        state = stm * state;
        setState(state);
        setCovariance(cov);

    }



}

```



## Step 4  Results 

This acceleration noise allow the estimation of the boundary to grow not only with velocity but also with acceleration. 

check the figure below to see how the figure, grow with acceleration. 
and with a slight change of the initial condition, we can end up with better estimation.



<div style="display: flex; justify-content: space-between;">
    <img src="figures/5.gif" width="400"/>
    <img src="figures/6.gif" width="400"/>
</div>




### Sumamry steps 1-4

- the initialization of the state allow the red circle to move from it's center with certain speed form an initial position
- covariance transition over time allow the boudnary of the dot to grow with certain speed, depending on the state of the covariance of the velocity.
- the error injected by acceleration, influence the estimation of position which might build up later with velocity giving the boundary of the circle the acceleartion you can see in the animation


## Step 5 - Update step using the GPS

All the previous steps resulted an increase of probability, making us more and more uncertain, the question right now is how to mitigate it? 

Answer is simple using measurement, gps, lidar, etc.

In this project we focus on measurement from the gps, information theory focuses on the difference between our estimations from mathmatical models and the actual data from the sensors.

the amount of surprise is described as innovation term.




$$ \text{Innovation} = y = z - H x_{estimate} $$

where H is a matrix that maps the full state into the measurement state, z represent the measured data from the sensors.

we also define a covariance in this innovation, which is basically the summation of the covariance of the estimated state and the coavraiance in the measurement of the data after making sure they have the same dimensions.


$$ \text{Innovation Covariance} = S = H p H^{T} + R $$

where H is a mapping matrix to ensure dimesnionality, R is error from the measurement process.


Finally we can reach the best estimate using the innovation, and innovation covaraince.

$$x^{+} = x^{-}+K y$$
$$P^{+} = (I - K H) P^{-}$$




